In [1]:
#| default_exp lafite_wrapper

In [2]:
#| exports

import sys ,os, subprocess
import argparse

from numpy import percentile
from time import strftime

from LAFITE.reference_processing import RefProcessWrapper, short_reads_sj_import, cage_tss_import
from LAFITE.preprocessing import read_grouping, polya_signal_import, PolyAFinder
from LAFITE.utils import temp_dir_creation, bam2bed, keep_tmp_file
from LAFITE.read_collapsing import CoCoWrapper
from LAFITE.tailFinder import TailFinderWrapper
from LAFITE.refine import RefineWrapper
from LAFITE.output import OutputAssembly

In [3]:
#| exports

class LafiteWrapper:
	def __init__(self, bam, bedtools, full_cleanup, gtf, genome, min_count_tss_tes, mis_intron_length, min_novel_trans_count, min_single_exon_coverage, min_single_exon_len, label, output, polya, polyA_motif_file, relative_abundance_threshold, sj_correction_window, short_sj_tab, thread, tss_cutoff, tss_peak):
		self.bam = bam
		self.bedtools = bedtools
		self.full_cleanup = full_cleanup
		self.gtf = gtf
		self.genome = genome
		self.min_count_tss_tes = min_count_tss_tes
		self.mis_intron_length = mis_intron_length
		self.min_novel_trans_count = min_novel_trans_count
		self.min_single_exon_coverage = min_single_exon_coverage
		self.min_single_exon_len = min_single_exon_len
		self.label = label
		self.output = output
		self.polya = polya
		self.polyA_motif_file = polyA_motif_file
		self.relative_abundance_threshold = relative_abundance_threshold
		self.sj_correction_window = sj_correction_window
		self.short_sj_tab = short_sj_tab
		self.thread = thread
		self.tss_cutoff = tss_cutoff
		self.tss_peak = tss_peak
		
	
	def revisit_parameter(self):
		"""
		revisit input parameters"""

		sys.stdout.write(f'\nInput parameters:\n')
		sys.stdout.write(f'Read alignment file: {self.bam}\n')
		sys.stdout.write(f'Reference gene annotation: {self.gtf}\n')
		sys.stdout.write(f'Reference genome annotation: {self.genome}\n')
		if self.polya:
			sys.stdout.write(f'Reads Polyadenylation events: {self.polya}\n')
		elif self.polyA_motif_file:
			sys.stdout.write(f'PolyA motif file for read Polyadenylation event estimation: {self.polyA_motif_file}\n')
		else:
			raise ValueError('Fatal: Please provide either polyA motif file or reads polyadenylation event result\n')
		sys.stdout.write(f'Edit distance to reference splicing site allowed for splicing correction: {self.sj_correction_window}\n')
		sys.stdout.write(f'Output assembly file: {self.output}\n')

	def run_lafite(self):
		"""
		LAFITE wrapper"""

		self.revisit_parameter()
		# create temp directory for log and intermediate files
		try:
			tmp_folder = temp_dir_creation(os.path.dirname(self.output))
			tmp_dir = tmp_folder.name
		except:
			raise ValueError('Fatal: Please provide a valid path for output files\n')

		# reference gene annotation processing
		sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': Preprocessing reference gene annotation\n')
		ref_exon, ref_junction, ref_single_exon_trans, ref_mutple_exon_trans, left_sj_set, right_sj_set, tss_dict = RefProcessWrapper(self.gtf, self.thread).result_collection()

		if self.short_sj_tab:
			left_sj_set, right_sj_set = short_reads_sj_import(self.short_sj_tab, left_sj_set, right_sj_set)
		
		if self.tss_peak:
			tss_dict = cage_tss_import(self.tss_peak, tss_dict)

		# processing alignment bam file
		## convert alignment bam file to bed12 format
		sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': Preprocessing alignment file\n')
		try: 
			bam2bed_cmd = bam2bed(self.bam, tmp_dir, self.bedtools)
		except:
			raise ValueError('Fatal: Please provide a valid path for bam file and bedtools\n')
		p = subprocess.run(bam2bed_cmd, shell=True)
		if p.returncode == 0:
			pass
		else:
			raise ValueError('Fatal: Error in bam conversion\n')

		## read grouping according to chromosome and strand
		outbed = os.path.join(tmp_dir, 'bam.bed')
		junction_dict, processed_read = read_grouping(outbed, self.genome)

		# polyA info import
		if self.polya:
			sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': Loading polyA information\n')
			polya_dict = polya_signal_import(self.polya)
		else:
			sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': No reads Polyadenylation event provided, detecting from sequence\n')
			polya_dict = PolyAFinder(processed_read, self.genome, self.polyA_motif_file).polya_estimation()

		# read correction and collapsing
		sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': Collapssing corrected reads\n')
		collected_single_exon_read, collected_multi_exon_read, collected_rss, collected_res = CoCoWrapper(self.thread, processed_read, ref_exon, ref_junction, ref_single_exon_trans, ref_mutple_exon_trans, left_sj_set, right_sj_set, junction_dict, self.sj_correction_window, polya_dict, self.mis_intron_length, tmp_dir).result_collection()

		# identify putative TSS and TES for collapsed reads
		processed_collected_multi_exon_read, three_prime_exon = TailFinderWrapper(collected_multi_exon_read, self.min_count_tss_tes, self.thread).result_collection()

		# calculating the tss_cutoff and tes_cutoff:
		if not self.tss_cutoff:
			self.tss_cutoff = percentile(collected_rss, 75)
		tes_cutoff = percentile(collected_res, 70)
		print(f'TSS cutoff: {self.tss_cutoff}')
		print(f'TES cutoff: {tes_cutoff}')

		# identify high quality isoforms from collapsed reads
		sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': Revisiting the collapsed reads to get high-concensus full-length isoforms\n')
		collected_refined_isoforms = RefineWrapper(processed_collected_multi_exon_read, collected_single_exon_read, ref_mutple_exon_trans, ref_single_exon_trans, three_prime_exon, tss_dict, self.tss_cutoff, tes_cutoff, self.min_novel_trans_count, self.min_single_exon_coverage, self.min_single_exon_len, self.thread, tmp_dir).result_collection()

		# output refined isoforms
		OutputAssembly(collected_refined_isoforms, self.output, self.label, self.relative_abundance_threshold).write_out()

		# keep intermediate results
		if self.full_cleanup:
			os.system(keep_tmp_file(self.output, tmp_dir))
		
		sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': All done!\n')

In [4]:
#| exports

def main():

	parser = argparse.ArgumentParser(description='Low-abundance Aware Full-length Isoform clusTEr')
	parser.add_argument('-b', dest='bam', help='path to the alignment file in bam format', required=True)
	parser.add_argument('-B', dest='bedtools', type=str, default='bedtools', help='path to the executable bedtools')
	parser.add_argument('-g', dest='gtf', help='path to the reference gene annotation in GTF format', required=True)
	parser.add_argument('-f', dest='genome', help='path to the reference genome fasta', required=True)
	parser.add_argument('-o', dest='output', help='path to the output file', required=True)
	parser.add_argument('-n', dest='min_count_tss_tes', type=int, default=3, help='minimum number of reads supporting a alternative TSS or TES, default: 3')
	parser.add_argument('-i', dest='mis_intron_length', type=int, default=150, help='length cutoff for correcting unexpected small intron, default: 150')
	parser.add_argument('-c', dest='min_novel_trans_count', type=int, default=3, help='minimum occurrences required for a isoform from novel loci, default: 3')
	parser.add_argument('-s', dest='min_single_exon_coverage', type=int, default=4, help='minimum read coverage required for a novel single-exon transcript, default: 4')
	parser.add_argument('-l', dest='min_single_exon_len', type=int, default=100, help='minimum length for single-exon transcript, default: 100')
	parser.add_argument('-L', dest='label', type=str, default='LAFT', help='name prefix for output transcripts, default: LAFT')
	parser.add_argument('-p', dest='polya', help='path to the file contains read Polyadenylation event')
	parser.add_argument('-m', dest='polyA_motif_file', help='path to the polya motif file')
	parser.add_argument('-r', dest='relative_abundance_threshold', type=int, default=0.01, help='minimum abundance of the predicted multi-exon transcripts as a fraction of the total transcript assembled at a given locus, default: 0.01')
	parser.add_argument('-j', dest='short_sj_tab', default=None, help='path to the short read splice junction file')
	parser.add_argument('-w', dest='sj_correction_window', type=int, default=40, help='edit distance to reference splicing site for splicing correction, default: 40')
	parser.add_argument('--no_full_cleanup', dest='full_cleanup', action='store_true', help='keep all intermediate files')
	parser.add_argument('-t', dest='thread', type=int, default=4, help='number of the threads, default: 4')
	parser.add_argument('-T', dest='tss_peak', default=None, help='path to the TSS peak file')
	parser.add_argument('-d', dest='tss_cutoff', type=int, default=None, help='minimum TSS distance for a transcript to be considered as a novel transcript')
	args = parser.parse_args()

	LafiteWrapper(args.bam, args.bedtools, args.full_cleanup, args.gtf, args.genome, args.min_count_tss_tes, args.mis_intron_length, args.min_novel_trans_count, args.min_single_exon_coverage, args.min_single_exon_len, args.label, args.output, args.polya, args.polyA_motif_file, args.relative_abundance_threshold, args.sj_correction_window, args.short_sj_tab, args.thread, args.tss_cutoff, args.tss_peak).run_lafite()


In [5]:

# gtf = '/expt/zjzace/Nanopore_subcellular/SIRV/SIRV_Set1/Raw_data/SIRV_isoforms_multi-fasta-annotation_C_170612a.gtf'
# bed = '/expt/zjzace/Nanopore_subcellular/SIRV/SIRV_Set1/bam/SRR6058583.sorted.bed'
# fa = '/expt/zjzace/Nanopore_subcellular/SIRV/SIRV_Set1/Raw_data/SIRV_isoforms_multi-fasta_170612a.fasta'
# junction_dict, processed_read = read_grouping(bed, fa)
# ref_exon, ref_junction, ref_single_exon_trans, ref_mutple_exon_trans, left_sj_set, right_sj_set, tss_dict = RefAnnotationExtraction(gtf).annotation_sorting()

# polya_dict = polya_signal_import('/expt/zjzace/Nanopore_subcellular/SIRV/SIRV_Set1/bam/SRR6058584.polya.res')
# polya_dict = PolyAFinder(processed_read, fa, '/home/zjzace/software/SQANTI3-4.1/data/polyA_motifs/mouse_and_human.polyA_motif.txt').polya_estimation()
# collected_single_exon_read, collected_multi_exon_read, tss, rss = CoCoWrapper(16, processed_read, ref_exon, ref_junction, ref_single_exon_trans, ref_mutple_exon_trans, left_sj_set, right_sj_set, junction_dict, tmp_dir='.', sj_correction_window=40, mis_intron_length = 150, corExcept_dis=4, polya_dict=polya_dict).result_collection()

In [6]:
#| hide

from nbdev import nbdev_export
nbdev_export()

In [7]:
output='test/tmp_reuslt.gtf'
gtf='test/ref.gtf'
genome='test/ref.fasta'
bam='test/test.bam'
polyA_motif_file='test/human_mouse_polyA_motif.txt'
thread=10
sj_correction_window=40
mis_intron_length=150
min_count_tss_tes=3

tmp_folder = temp_dir_creation(os.path.dirname(output))
tmp_dir = tmp_folder.name
sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': Preprocessing reference gene annotation\n')
ref_exon, ref_junction, ref_single_exon_trans, ref_mutple_exon_trans, left_sj_set, right_sj_set, tss_dict = RefProcessWrapper(gtf, thread).result_collection()

bam2bed_cmd = bam2bed(bam, tmp_dir, 'bedtools')

p = subprocess.run(bam2bed_cmd, shell=True)
if p.returncode == 0:
    pass
else:
    raise ValueError('Fatal: Error in bam conversion\n')

outbed = os.path.join(tmp_dir, 'bam.bed')
junction_dict, processed_read = read_grouping(outbed, genome)

sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': No reads Polyadenylation event provided, detecting from sequence\n')
polya_dict = PolyAFinder(processed_read, genome, polyA_motif_file).polya_estimation()

# read correction and collapsing
sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': Collapssing corrected reads\n')
collected_single_exon_read, collected_multi_exon_read, collected_rss, collected_res = CoCoWrapper(thread, processed_read, ref_exon, ref_junction, ref_single_exon_trans, ref_mutple_exon_trans, 
                                                                                                  left_sj_set, right_sj_set, junction_dict, sj_correction_window, polya_dict, mis_intron_length, tmp_dir).result_collection()
processed_collected_multi_exon_read, three_prime_exon = TailFinderWrapper(collected_multi_exon_read, min_count_tss_tes, thread).result_collection()
tss_cutoff = percentile(collected_rss, 75)
tes_cutoff = percentile(collected_res, 70)
print(f'TSS cutoff: {tss_cutoff}')
print(f'TES cutoff: {tes_cutoff}')
sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': Revisiting the collapsed reads to get high-concensus full-length isoforms\n')

<TemporaryDirectory 'test/tmpip86zkd2'>
2023-08-23 16:36:13: Preprocessing reference gene annotation


2023-08-23 16:36:15: No reads Polyadenylation event provided, detecting from sequence
2023-08-23 16:36:15: Collapssing corrected reads


2023-08-23 16:36:15: Collapsing raw reads from SIRV3 -:   0%|          | 0/2164 [00:00<?, ?it/s]0237.34it/s]
2023-08-23 16:36:15: Collapsing raw reads from SIRV5 -: 100%|██████████| 61/61 [00:00<00:00, 7632.60it/s]s]

2023-08-23 16:36:16: Collapsing raw reads from SIRV6 -: 100%|██████████| 466/466 [00:00<00:00, 4944.09it/s]s]
2023-08-23 16:36:15: Collapsing raw reads from SIRV4 +:  78%|███████▊  | 1956/2509 [00:00<00:00, 7467.30it/s]]
2023-08-23 16:36:15: Collapsing raw reads from SIRV3 -:  88%|████████▊ | 1895/2164 [00:00<00:00, 7625.65it/s]
2023-08-23 16:36:15: Collapsing raw reads from SIRV3 +:  29%|██▉       | 1187/4048 [00:00<00:00, 4638.75it/s]]
2023-08-23 16:36:16: Collapsing raw reads from SIRV6 +:  33%|███▎      | 3755/11292 [00:00<00:00, 37547.35it/s]
2023-08-23 16:36:16: Collapsing raw reads from SIRV6 +:   0%|          | 0/11292 [00:00<?, ?it/s]

2023-08-23 16:36:15: Collapsing raw reads from SIRV3 +: 100%|██████████| 4048/4048 [00:00<00:00, 12983.02it/s]
2023-08-23 16:36:1

TSS cutoff: 31.0
TES cutoff: 16.0
2023-08-23 16:36:26: Revisiting the collapsed reads to get high-concensus full-length isoforms


95

In [8]:

min_single_exon_coverage=4
min_novel_trans_count=3
min_single_exon_len=100
label='LFT'
relative_abundance_threshold=0.01
# identify putative TSS and TES for collapsed reads



# identify high quality isoforms from collapsed reads
sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': Revisiting the collapsed reads to get high-concensus full-length isoforms\n')
collected_refined_isoforms = RefineWrapper(processed_collected_multi_exon_read, collected_single_exon_read, ref_mutple_exon_trans, ref_single_exon_trans, three_prime_exon, 
                                           tss_dict, tss_cutoff, tes_cutoff,min_novel_trans_count, min_single_exon_coverage, min_single_exon_len, thread, tmp_dir).result_collection()

# output refined isoforms
OutputAssembly(collected_refined_isoforms, output, label, relative_abundance_threshold).write_out()

# keep intermediate results
# if self.full_cleanup:
#     os.system(keep_tmp_file(self.output, tmp_dir))

sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ': All done!\n')

2023-08-23 16:36:26: Revisiting the collapsed reads to get high-concensus full-length isoforms
full_block      (10599, 10791, 10898, 11187, 11404, 11606)
start                                                10599
end                                                  11606
count                                                  145
polya_count                                             24
fsm                                                   True
polyaed                                              False
as_site                                     [10599, 10654]
apa_site                                    [11606, 11542]
name                                           SIRV1_POS.1
reference_id                               [SIRV108, None]
rss_dis                                                 16
read_tag                                         Reference
processed                                             True
chrand_ID                                                1
abundance           

31

In [9]:
collected_refined_isoforms

{('SIRV1',
  '+'): {(10599,
   10791,
   10898,
   11187,
   11404,
   11606): AttributeCollection(start=10599, end=11606, count=145, polya_count=24, fsm=True, polyaed=False, as_site=[10599, 10654], apa_site=[11606, 11542], name='SIRV1_POS.1', reference_id=['SIRV108', None], rss_dis=16, read_tag='Reference', processed=True, chrand_ID=1, loci_ID=1), (10723,
   10791,
   10883,
   11057,
   11435,
   11639): AttributeCollection(start=10723, end=11639, count=26, polya_count=0, fsm=True, polyaed=False, as_site=[10723], apa_site=[11639], name='SIRV1_POS.4', reference_id=[None, 'SIRV109'], rss_dis=11, read_tag='Reference', processed=True, chrand_ID=1, loci_ID=1)},
 ('SIRV1',
  '-'): {(1024,
   1484,
   6338,
   6473,
   6561,
   6813,
   7553,
   7814,
   10283,
   10366,
   10648,
   10774): AttributeCollection(start=10774, end=1024, count=169, polya_count=0, fsm=True, polyaed=False, as_site=[10774], apa_site=[1024, 1068], name='SIRV1_NEG.3', reference_id=['SIRV103', None], rss_dis=12, read

In [10]:
from collections import defaultdict
from interlap import InterLap
import os
# from pyfastx import Fasta



In [11]:


class Vividict(dict):
	"""create nest dictionary
	"""
	def __missing__(self, key):
		value = self[key] = type(self)()
		return value

def gtf_line_split (line):
	line = line.rstrip().split('\t')
	chrom, source, feature, start, end, score, strand, frame, attribute = line
	start = int(start)
	end = int(end)
	return chrom, source, feature, start, end, score, strand, frame, attribute

def gtf2splicing(gtf, transcript_ref = 'transcript_id'):
	"""preprocess the gtf file to isoform level
	"""
	isoform_structure_dict = Vividict()
	with open (gtf) as f:
		for line in f:
			if not line.startswith('#'):
				chrom, source, feature, start, end, score, strand, frame, attributes = gtf_line_split(line)
				try:
					attributes = [a.strip().replace('"','') for a in attributes.strip(';').split('"; ') if len(a)>0]
					attributes = dict([a.split(' ',1)[0:2] for a in attributes])
				except:
					raise ValueError("Fatal: please check input GTF format\n")
				if transcript_ref in attributes:
					transcript_id  = attributes[transcript_ref]
					if transcript_id not in isoform_structure_dict[(chrom, strand)]:
						isoform_structure_dict[(chrom, strand)][transcript_id] = [None, []]
	
					if feature in ('transcript', 'mRNA'):
						transcript_id  = attributes[transcript_ref]
						isoform_structure_dict[(chrom, strand)][transcript_id][0] = attributes
					if feature == "exon":
						isoform_structure_dict[(chrom, strand)][transcript_id][1].extend([start,end])

	return isoform_structure_dict

def annotation_reshape(isoform_structure_dict):
	reshaped_multi_exon_isoform_dict = Vividict()
	reshaped_single_exon_isoform_dict = Vividict()
	single_exon_isoform_interlap = defaultdict(set) 

	for chr_str, transcript in isoform_structure_dict.items():
		chrom, strand = chr_str
		for transcript_id, isoform_detail in transcript.items():
			attributes, isoform_structure = isoform_detail
			isoform_structure.sort()
			splicing = isoform_structure[1:-1]
			if splicing:
				reshaped_multi_exon_isoform_dict[(chrom, strand)][tuple(splicing)] = [transcript_id, isoform_structure[0], isoform_structure[-1], attributes]
			else:
				single_exon_isoform_interlap[(chrom, strand)].add(tuple(isoform_structure))
				reshaped_single_exon_isoform_dict[(chrom, strand)][tuple(isoform_structure)] = [transcript_id, isoform_structure[0], isoform_structure[-1], attributes]
	
	# convert to interlap 
	for i in single_exon_isoform_interlap:
		t = list(single_exon_isoform_interlap[i])
		single_exon_isoform_interlap[i] = InterLap()
		single_exon_isoform_interlap[i].update(t)
	return reshaped_multi_exon_isoform_dict, reshaped_single_exon_isoform_dict, single_exon_isoform_interlap


def split_bed_line(bed_line,extends=False):
	"""
	split bed line
	"""
	cells = bed_line.strip('\n').split("\t")
	chrom = start = end = name = score = strand = thick_start = thick_end = item_rgb = block_count = block_sizes = block_starts = others = None
	if len(cells) >= 3:
		chrom = cells[0]
		start = int(cells[1])
		end = int(cells[2])
	if len(cells) >=6:
		name = cells[3]
		score = cells[4]
		strand= cells[5]
	if len(cells) >=12:
		thick_start = int(cells[6])
		thick_end = int(cells[7])
		item_rgb = cells[8]
		block_count = int(cells[9])
		block_sizes = [int(i) for i in cells[10].rstrip(',').split(',')]
		block_starts= [int(i) for i in cells[11].rstrip(',').split(',')]
		if extends:
			others = cells[12:]
			return chrom,start,end,name,score,strand,thick_start,thick_end,item_rgb,block_count,block_sizes,block_starts,others
	return chrom,start,end,name,score,strand,thick_start,thick_end,item_rgb,block_count,block_sizes,block_starts
	

def bed_block_to_splicing(start, block_count, block_starts, block_sizes):
	read_splicing = []
	if block_count > 1:
		for i in range(block_count-1):
			left_sj = start + block_starts[i] + block_sizes[i]
			right_sj = start + block_starts[i+1] + 1
			read_splicing.extend([left_sj, right_sj])
	return read_splicing

def read_assignment(bed, reshaped_multi_exon_isoform_dict, reshaped_single_exon_isoform_dict, single_exon_isoform_interlap, ratio=0.8):
	cp_read_dict = defaultdict(list)
	with open(bed) as f:
		for line in f:
			chrom,start,end,name,score,strand,thick_start,thick_end,item_rgb,block_count,block_sizes,block_starts = split_bed_line(line)
			if block_count > 1:
				read_splicing = tuple(bed_block_to_splicing(start, block_count, block_starts, block_sizes))
				if read_splicing in reshaped_multi_exon_isoform_dict[(chrom, strand)]:
					transcript = reshaped_multi_exon_isoform_dict[(chrom, strand)][read_splicing][0]
					cp_read_dict[transcript].append(name)
			elif single_exon_isoform_interlap[(chrom, strand)]:
				overlap_gene = list(single_exon_isoform_interlap[(chrom, strand)].find((start+1,end)))
				if overlap_gene:
					for i in overlap_gene:
						transcript = reshaped_single_exon_isoform_dict[(chrom, strand)][(i[0],i[1])][0]
						inter = list(set(range(start,end+1)).intersection(range(i[0],i[1]+1)))
						inter.sort()
						if (inter[-1]-inter[0]+1)/(i[1]-i[0]+1) > ratio:
							cp_read_dict[transcript].append(name)
	return cp_read_dict

def read_path_process(file):
	read_path = {}
	with open(file) as f:
		for read in f:
			read = read.rstrip()
			name = os.path.basename(read).split('.')[0]
			read_path[name] = read
	return read_path

In [12]:
reshaped_multi_exon_isoform_dict, reshaped_single_exon_isoform_dict, single_exon_isoform_interlap = annotation_reshape(gtf2splicing('test/results'))

In [13]:
cp_read_dict=read_assignment('test/tmppuw1rf8t/Corrected_reads.bed', reshaped_multi_exon_isoform_dict, reshaped_single_exon_isoform_dict, single_exon_isoform_interlap)

FileNotFoundError: [Errno 2] No such file or directory: 'test/tmppuw1rf8t/Corrected_reads.bed'

In [ ]:
cp_read_dict

defaultdict(list,
            {'LFT.2.3': ['SRR6058583.300425',
              'SRR6058583.382',
              'SRR6058583.1721',
              'SRR6058583.38823',
              'SRR6058583.46769',
              'SRR6058583.49841',
              'SRR6058583.52892',
              'SRR6058583.87685',
              'SRR6058583.95002',
              'SRR6058583.106472',
              'SRR6058583.120438',
              'SRR6058583.120459',
              'SRR6058583.135800',
              'SRR6058583.201486',
              'SRR6058583.219568',
              'SRR6058583.230474',
              'SRR6058583.242601',
              'SRR6058583.264816',
              'SRR6058583.265139',
              'SRR6058583.14195',
              'SRR6058583.156937',
              'SRR6058583.159524',
              'SRR6058583.223720',
              'SRR6058583.269490',
              'SRR6058583.9440',
              'SRR6058583.18655',
              'SRR6058583.31543',
              'SRR6058583.40178',
        

In [ ]:
########################################This script is to allocate the nanopore reads to each transcript #######################################
############################This script also copy the transcript fasta to the corresponding transcript directory################################





def run_cp(read_path, assign_cmd, outdir, transcriptome, cp_read_dict):
	fasta_sequences = Fasta(transcriptome)
	with open(assign_cmd, 'w') as fw:
		for transcript, reads in cp_read_dict.items():
			if len(reads) >= 3:
				outFile = f'{outdir}/{transcript}/fasta.fa'
				outFolder = f'{outdir}/{transcript}/fast5'

				if os.path.exists(outFolder) == False:
					os.system(f'mkdir -p {outFolder}')
				if os.path.exists(outFile) == False:
					with open(outFile, 'w') as faw:
						fa = fasta_sequences[transcript].seq
						faw.write(f'>{transcript}\n{fa}\n')
				for read in reads:
					cmd = f'cp {read_path[read]} {outdir}/{transcript}/fast5'
					fw.write(cmd+'\n')

if __name__ == '__main__':
	parser = argparse.ArgumentParser(description='This script is to allocate the nanopore reads to each transcript')
	parser.add_argument('-b', '--bed', help='The bed file of corrected reads')
	parser.add_argument('-o', '--outdir', help='The output directory of the re-assigned reads')
	parser.add_argument('-c', '--cmd', help='The output file of the copy read commands')
	parser.add_argument('-f', '--transcriptome', help='The transcriptome fasta file')
	parser.add_argument('-r', '--read_path', help='The collection of the path of the splited nanopore reads')
	parser.add_argument('-g', '--gtf', help='The gtf file of the transcriptome')
	
	args = parser.parse_args()

	reshaped_multi_exon_isoform_dict, reshaped_single_exon_isoform_dict, single_exon_isoform_interlap = annotation_reshape(gtf2splicing(args.gtf))
	read_path = read_path_process(args.read_path)
	cp_read_dict = read_assignment(args.bed, reshaped_multi_exon_isoform_dict, reshaped_single_exon_isoform_dict, single_exon_isoform_interlap)
	run_cp(read_path, args.cmd, args.outdir, args.transcriptome, cp_read_dict)



In [ ]:

gtf = '/expt/zjzace/Nanopore_subcellular/SIRV/SIRV_Set1/Raw_data/SIRV_isoforms_multi-fasta-annotation_C_170612a.gtf'
bed = '/expt/zjzace/Nanopore_subcellular/SIRV/SIRV_Set1/bam/SRR6058583.sorted.bed'
fa = '/expt/zjzace/Nanopore_subcellular/SIRV/SIRV_Set1/Raw_data/SIRV_isoforms_multi-fasta_170612a.fasta'
junction_dict, processed_read = read_grouping(bed, fa)
ref_exon, ref_junction, ref_single_exon_trans, ref_mutple_exon_trans, left_sj_set, right_sj_set, tss_dict = RefAnnotationExtraction(gtf).annotation_sorting()

# polya_dict = polya_signal_import('/expt/zjzace/Nanopore_subcellular/SIRV/SIRV_Set1/bam/SRR6058584.polya.res')
# polya_dict = PolyAFinder(processed_read, fa, '/home/zjzace/software/SQANTI3-4.1/data/polyA_motifs/mouse_and_human.polyA_motif.txt').polya_estimation()
# collected_single_exon_read, collected_multi_exon_read, tss, rss = CoCoWrapper(16, processed_read, ref_exon, ref_junction, ref_single_exon_trans, ref_mutple_exon_trans, left_sj_set, right_sj_set, junction_dict, tmp_dir='.', sj_correction_window=40, mis_intron_length = 150, corExcept_dis=4, polya_dict=polya_dict).result_collection()